In [1]:
import requests
import pandas as pd
import re
import pickle
import os
import warnings
from tqdm import tnrange, tqdm_notebook
from PIL import Image

In [5]:
url_tpl = "https://unsplash.com/napi/search/photos?query=full%20hd&xp=&per_page=300&page={}"
directory = "photo"

# Find image URLs

In [80]:
page = 1
all_photo_df = pd.DataFrame()

while True:
    url = url_tpl.format(page)
    resp = requests.get(url = url)
    photo_list = resp.json()["results"]
    if len(photo_list) == 0:
        break
    photo_df = pd.DataFrame(photo_list)
    all_photo_df = pd.concat([all_photo_df, photo_df])
    page = page + 1
    
all_photo_df = all_photo_df.reset_index(drop=True)

In [140]:
all_photo_df_mini = pd.DataFrame({
    'width': all_photo_df['width'],
    'height': all_photo_df['height'],
    'url': all_photo_df['urls'].apply(lambda x : x['raw'])
})

In [148]:
all_photo_file = open('all_photo_df.pkl', 'wb')
pickle.dump(all_photo_df_mini, all_photo_file)
all_photo_file.close()

# Download images

In [3]:
all_photo_file = open('all_photo_df.pkl', 'rb')
all_photo_df_mini = pickle.load(all_photo_file)
all_photo_file.close()

In [ ]:
num = 0
location_tpl = "../images/photo_raw/%04d_%04d_%04d.jpg"

for id, photo in all_photo_df_mini.iterrows():
    width = photo['width']
    height = photo['height']
    im_url = photo['url']
    location = location_tpl % (num, width, height)
    if not os.path.isfile(location):
        img = requests.get(im_url).content
        with open(location, 'wb') as f:
            f.write(img)
    num = num + 1

# Convert images

## HD conversion

In [2]:
input_dir = "../images/photo_raw"
output_dir = "../images/photo_fullhd"

target_width = 1920
target_height = 1080

image_files = os.listdir(input_dir)
image_files = list(filter(lambda x: x.endswith('.jpg'), image_files))

Image.MAX_IMAGE_PIXELS = 999999999999 # cause there are some large files

for im in tqdm_notebook(image_files):
    input_loc = input_dir + '/' + im
    output_loc = output_dir + '/' + im

    img = Image.open(input_loc)
    img = img.convert('RGB') # as it seems some images are not JPG

    if img.width < img.height:
        img = img.rotate(90, expand=1)

    scale = max(target_width / img.width, target_height / img.height)
    new_width = round(scale * img.width)
    new_height = round(scale * img.height)

    img_resized = img.resize((new_width, new_height), Image.LANCZOS)

    crop_left = int((img_resized.width - target_width)/2)
    crop_top = int((img_resized.height - target_height)/2)

    img_cropped = img_resized.crop((crop_left, crop_top, crop_left + target_width, crop_top + target_height))

    img_cropped.save(output_loc, quality = 100)

KeyboardInterrupt: 

## Lowres conversion

In [7]:
input_dir = "../images/photo_raw"
output_dir = "../images/photo_lowres"

target_width = 480
target_height = 270

image_files = os.listdir(input_dir)
image_files = list(filter(lambda x: x.endswith('.jpg'), image_files))

Image.MAX_IMAGE_PIXELS = 999999999999 # cause there are some large files

for im in tqdm_notebook(image_files):
    input_loc = input_dir + '/' + im
    output_loc = output_dir + '/' + im

    img = Image.open(input_loc)
    img = img.convert('RGB') # as it seems some images are not JPG

    if img.width < img.height:
        img = img.rotate(90, expand=1)

    scale = max(target_width / img.width, target_height / img.height)
    new_width = round(scale * img.width)
    new_height = round(scale * img.height)

    img_resized = img.resize((new_width, new_height), Image.LANCZOS)

    crop_left = int((img_resized.width - target_width)/2)
    crop_top = int((img_resized.height - target_height)/2)

    img_cropped = img_resized.crop((crop_left, crop_top, crop_left + target_width, crop_top + target_height))

    img_cropped.save(output_loc, quality = 100)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
